In [29]:
import torch
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments, GPT2Config
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [30]:
# Check if a GPU is available and set PyTorch to use the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [31]:
configuration = GPT2Config()
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token to be the eos_token
tokenizer.pad_token = tokenizer.eos_token

# Load GPT-2 model
model = GPT2ForSequenceClassification(configuration).from_pretrained('gpt2', num_labels = 3).to(device)
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
# Load the dataset
df = pd.read_csv('ucberkeley_measuring_hate_speech_dataset_training.csv')

df = df.sample(frac=1).reset_index(drop=True)

balanced_df = pd.concat([df[df['hatespeech'] == label].head(120) for label in range(3)])

balanced_df

train_df, val_df = train_test_split(balanced_df, test_size=0.2)

In [33]:
class HateSpeechDataset(Dataset):
    def __init__(self, dataframe):
        self.encodings = tokenizer(dataframe['text'].tolist(), truncation=True, padding=True, max_length=128)
        self.labels = dataframe['hatespeech'].tolist()  # Ensure this is a list of integers

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Ensure labels are torch.long type
        return item

    def __len__(self):
        return len(self.labels)


In [34]:
class TestDataset(Dataset):
    def __init__(self, dataframe):
        self.encodings = tokenizer(dataframe['text'].tolist(), truncation=True, padding=True, max_length=128)

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [35]:
train_dataset = HateSpeechDataset(train_df)
val_dataset = HateSpeechDataset(val_df)

In [36]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=16,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
)

In [37]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [38]:
# Train the model
trainer.train()

# Save the model
model.save_pretrained("./trained_model")

  0%|          | 0/108 [00:00<?, ?it/s]

{'train_runtime': 14.9053, 'train_samples_per_second': 57.966, 'train_steps_per_second': 7.246, 'train_loss': 3.6333383630823204, 'epoch': 3.0}


In [39]:
test_df = pd.read_csv("ucberkeley_measuring_hate_speech_dataset_testing.csv")
test_dataset = TestDataset(test_df)

In [40]:
# Load trained model
model = GPT2ForSequenceClassification.from_pretrained("./trained_model")
model.eval()

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=3, bias=False)
)

In [41]:
# DataLoader for test dataset
test_loader = DataLoader(test_dataset, batch_size=8)

# Make predictions
predictions = []
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.tolist())

In [42]:
# Save final predictions
test_df['hatespeech'] = predictions
test_df.to_csv("GPT2_sequential_clf_final_predictions.csv", index=False)